In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
from gnatss.ops.posfilter import kalman_filtering
from gnatss.loaders import read_novatel_L1_data_files
from gnatss.loaders import load_gps_solutions, load_travel_times
from gnatss.ops.data import clean_tt, preprocess_tt
from gnatss import constants

import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.5f' % x)


import warnings
warnings.filterwarnings('ignore')

In [3]:
travel_time_files = ['tests/data/2022/NCL1/208/WG_20220727/pxp_tt', 'tests/data/2022/NCL1/209/WG_20220728/pxp_tt', 'tests/data/2022/NCL1/210/WG_20220729/pxp_tt']

transponder_ids = ['NCL1-1', 'NCL1-2', 'NCL1-3']
travel_times_correction = 0.0
transducer_delay_time = 0.0

all_travel_times = load_travel_times(
        files=travel_time_files, transponder_ids=transponder_ids
    )


cleaned_travel_times = clean_tt(all_travel_times, transponder_ids, travel_times_correction, transducer_delay_time)
twtt_df = preprocess_tt(cleaned_travel_times)
# twtt = twtt_df.rename(columns={'time': 'dts'})
twtt_df

,time,MT,TT,ST
0,712215465.00000,ST,0.00000,NaN
1,712215467.26326,NCL1-1,2.26326,712215465.00000
2,712215467.38340,NCL1-2,2.38340,712215465.00000
3,712215467.40082,NCL1-3,2.40082,712215465.00000
4,712215480.00000,ST,0.00000,NaN
...,...,...,...,...
34407,712396373.47335,NCL1-3,2.47334,712396371.00000
34408,712396386.00000,ST,0.00000,NaN
34409,712396388.25092,NCL1-1,2.25093,712396386.00000
34410,712396388.31850,NCL1-2,2.31850,712396386.00000


In [4]:
inspvaa_df = read_novatel_L1_data_files(data_files=['tests/data/2022/NCL1/NCL1_INSPVAA.dat'], data_format="INSPVAA")
insstdeva_df = read_novatel_L1_data_files(data_files=['tests/data/2022/NCL1/NCL1_INSSTDEVA.dat'],
                                          data_format="INSSTDEVA")

In [5]:
inspvaa_df

,Week,Seconds,lat,lon,alt,ant_n,ant_e,ant_u,roll,pitch,heading,time
0,2220,400732.75000,45.30228,-124.96597,-28.34490,-0.68580,0.10140,0.24130,-3.55017,1.67569,225.48596,712293532.75000
1,2220,400732.80000,45.30228,-124.96597,-28.33400,-0.63800,0.07880,0.27340,-3.54066,1.37273,225.57350,712293532.80000
2,2220,400732.85000,45.30228,-124.96597,-28.32250,-0.58330,0.05340,0.29260,-3.18302,1.12580,225.64909,712293532.85000
3,2220,400732.90000,45.30228,-124.96597,-28.31040,-0.54400,0.04050,0.29500,-2.58513,0.94125,225.70457,712293532.90000
4,2220,400732.95000,45.30228,-124.96597,-28.29900,-0.51220,0.03460,0.28010,-1.81905,0.82661,225.73288,712293532.95000
...,...,...,...,...,...,...,...,...,...,...,...,...
3865082,2220,469048.15000,45.30206,-124.96584,-23.49010,-0.97260,1.06310,-0.11580,2.40545,-1.05323,127.18168,712361848.15000
3865083,2220,469048.20000,45.30206,-124.96584,-23.49620,-0.98040,1.08850,-0.14160,2.84893,-0.73574,126.52136,712361848.20000
3865084,2220,469048.25000,45.30205,-124.96584,-23.50370,-0.97090,1.12550,-0.17280,3.10908,-0.44089,125.87306,712361848.25000
3865085,2220,469048.30000,45.30205,-124.96584,-23.51290,-0.95420,1.15780,-0.20870,3.18417,-0.16181,125.24787,712361848.30000


In [6]:
insstdeva_df

,Week,Seconds,lat std,lon std,alt std,ant_n std,ant_e std,ant_u std,roll_roll,pitch_pitch,heading_heading,Ext sol stat,Time Since Update,time
0,2220,400732.75000,0.50020,0.44000,0.72830,0.01790,0.01590,0.01600,0.03010,0.03080,0.14650,0e0000d3,0,712293532.75000
1,2220,400732.80000,0.50020,0.44000,0.72830,0.01790,0.01590,0.01600,0.03010,0.03080,0.14650,0e0000d3,0,712293532.80000
2,2220,400732.85000,0.50020,0.44000,0.72830,0.01790,0.01590,0.01600,0.03010,0.03080,0.14650,0e0000d3,0,712293532.85000
3,2220,400732.90000,0.50020,0.44000,0.72830,0.01790,0.01590,0.01600,0.03010,0.03080,0.14650,0e0000d3,0,712293532.90000
4,2220,400732.95000,0.50020,0.44000,0.72830,0.01790,0.01590,0.01600,0.03010,0.03080,0.14650,0e0000d3,0,712293532.95000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3865303,2220,469048.15000,0.37480,0.31840,0.74050,0.01720,0.01470,0.02060,0.02520,0.02570,0.12630,0e0000d3,0,712361848.15000
3865304,2220,469048.20000,0.37480,0.31840,0.74050,0.01720,0.01470,0.02060,0.02520,0.02570,0.12630,0e0000d3,0,712361848.20000
3865305,2220,469048.25000,0.37480,0.31840,0.74050,0.01720,0.01470,0.02060,0.02520,0.02570,0.12630,0e0000d3,0,712361848.25000
3865306,2220,469048.30000,0.37480,0.31840,0.74050,0.01720,0.01470,0.02060,0.02520,0.02570,0.12630,0e0000d3,0,712361848.30000


In [7]:
gps_novatel_df = load_gps_solutions(
    files = [
        'tests/data/2022/NCL1/208/GPS_PPP/GPS_POS_FREED',
        'tests/data/2022/NCL1/209/GPS_PPP/GPS_POS_FREED',
        'tests/data/2022/NCL1/210/GPS_PPP/GPS_POS_FREED',
    ],
    is_l1_data=True,
)
gps_novatel_df

,time,x,y,z,sdx,sdy,sdz
0,712207183.00000,-2574893.32050,-3681574.16020,4512096.64040,2.44910,2.44890,2.44880
1,712207184.00000,-2574893.27980,-3681574.05620,4512096.74780,2.23590,2.23570,2.23560
2,712207185.00000,-2574893.23910,-3681573.95220,4512096.85520,2.00000,1.99980,1.99970
3,712207186.00000,-2574893.19830,-3681573.84820,4512096.96260,1.73230,1.73200,1.73190
4,712207187.00000,-2574893.15760,-3681573.74410,4512097.07000,1.41470,1.41440,1.41420
...,...,...,...,...,...,...,...
193288,712400473.00000,-2575282.93430,-3682610.17990,4511037.84700,0.01340,0.01370,0.01580
193289,712400474.00000,-2575283.67510,-3682610.14460,4511038.07690,0.01340,0.01370,0.01580
193290,712400475.00000,-2575283.93220,-3682610.22220,4511037.95340,0.01340,0.01370,0.01580
193291,712400476.00000,-2575283.70300,-3682609.18440,4511037.48930,0.01340,0.01370,0.01580


In [9]:
kalman_filtered_df, x, P, K, Pp = kalman_filtering(inspvaa_df, insstdeva_df, gps_novatel_df, twtt_df)

run_filter_simulation with parameters: gnss_pos_psd=3.125e-05, vel_psd=0.0025, cov_err=0.25


In [13]:
kalman_filtered_df = kalman_filtered_df[
    ['time', *constants.ANT_GPS_GEOCENTRIC, *constants.ANT_GPS_COV]
]
kalman_filtered_df

,time,ant_x,ant_y,ant_z,ant_xx,ant_xy,ant_xz,ant_yx,ant_yy,ant_yz,ant_zx,ant_zy,ant_zz
0,712215465.00000,-2575288.61527,-3682570.76041,4511064.96091,0.00008,0.00000,-0.00000,0.00000,0.00025,-0.00000,-0.00000,-0.00000,0.00017
1,712215467.26326,-2575287.15687,-3682572.08360,4511065.66541,0.00020,0.00000,-0.00000,0.00000,0.00033,-0.00000,-0.00000,-0.00000,0.00026
2,712215467.38340,-2575287.09816,-3682572.13761,4511065.66233,0.00022,0.00000,-0.00000,0.00000,0.00034,-0.00000,-0.00000,-0.00000,0.00029
3,712215467.40082,-2575287.08863,-3682572.14487,4511065.66017,0.00023,0.00000,-0.00000,0.00000,0.00035,-0.00000,-0.00000,-0.00000,0.00029
4,712215480.00000,-2575279.82059,-3682576.56931,4511066.24989,0.00008,0.00000,-0.00000,0.00000,0.00025,-0.00000,-0.00000,-0.00000,0.00017
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34407,712396373.47335,-2575339.00177,-3682613.25255,4511003.33026,0.00023,0.00000,-0.00000,0.00000,0.00025,-0.00000,-0.00000,-0.00000,0.00029
34408,712396386.00000,-2575336.64477,-3682622.46880,4510997.33401,0.00008,0.00000,-0.00000,0.00000,0.00011,-0.00000,-0.00000,-0.00000,0.00018
34409,712396388.25092,-2575336.76729,-3682622.83128,4510996.26970,0.00019,0.00000,-0.00000,0.00000,0.00022,-0.00000,-0.00000,-0.00000,0.00027
34410,712396388.31850,-2575336.74205,-3682622.86491,4510996.26623,0.00022,0.00000,-0.00000,0.00000,0.00024,-0.00000,-0.00000,-0.00000,0.00028


In [15]:
kalman_filtered_df.columns = ["time", "x", "y", "z", 'xx', 'xy', 'xz', 'yx', 'yy', 'yz', 'zx', 'zy', 'zz']

In [14]:
gps_pos_freed_trans = load_gps_solutions(
    files = [
        'tests/data/POS_TWTT',
        # 'tests/data/2022/NCL1/209/posfilter/POS_FREED_TRANS_TWTT',
        # 'tests/data/2022/NCL1/210/posfilter/POS_FREED_TRANS_TWTT',
    ],
)

gps_pos_freed_trans = gps_pos_freed_trans[
    ["time", "x", "y", "z", 'xx', 'xy', 'xz', 'yx', 'yy', 'yz', 'zx', 'zy', 'zz']
]
gps_pos_freed_trans

,time,x,y,z,xx,xy,xz,yx,yy,yz,zx,zy,zz
0,712215465.00000,-2575288.61500,-3682570.76100,4511064.96100,0.00076,0.00000,-0.00000,0.00000,0.00100,-0.00000,-0.00000,-0.00000,0.00089
1,712215467.26326,-2575287.15700,-3682572.08300,4511065.66500,0.00032,0.00000,-0.00000,0.00000,0.00055,-0.00000,-0.00000,-0.00000,0.00044
2,712215467.38340,-2575287.09800,-3682572.13700,4511065.66200,0.00041,0.00000,-0.00000,0.00000,0.00065,-0.00000,-0.00000,-0.00000,0.00054
3,712215467.40082,-2575287.08900,-3682572.14500,4511065.66000,0.00048,0.00000,-0.00000,0.00000,0.00072,-0.00000,-0.00000,-0.00000,0.00060
4,712215480.00000,-2575279.82100,-3682576.57000,4511066.25000,0.00076,0.00000,-0.00000,0.00000,0.00100,-0.00000,-0.00000,-0.00000,0.00089
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4447,712238383.29579,-2576531.23400,-3682656.04000,4510291.03600,0.00044,-0.00000,-0.00000,-0.00000,0.00042,-0.00000,-0.00000,-0.00000,0.00038
4448,712238383.91264,-2576531.70900,-3682655.55000,4510290.96900,0.00088,-0.00000,-0.00000,-0.00000,0.00087,-0.00000,-0.00000,-0.00000,0.00083
4449,712238396.00000,-2576537.79700,-3682654.09700,4510289.00000,0.00088,-0.00000,-0.00000,-0.00000,0.00087,-0.00000,-0.00000,-0.00000,0.00083
4450,712238398.29877,-2576539.92100,-3682653.44700,4510289.00500,0.00044,-0.00000,-0.00000,-0.00000,0.00043,-0.00000,-0.00000,-0.00000,0.00043


In [17]:
pd.set_option('display.float_format', lambda x: '%.8f' % x)

common_df = kalman_filtered_df.merge(
    gps_pos_freed_trans,
    on="time",
    how="inner",
    suffixes=("_kalman", "_gold")
)
common_df = common_df[[
    'time','x_kalman', 'x_gold','y_kalman', 'y_gold','z_kalman', 'z_gold',
    'xx_kalman', 'xx_gold', 'xy_kalman', 'xy_gold', 'xz_kalman', 'xz_gold',
    'yx_kalman', 'yx_gold', 'yy_kalman', 'yy_gold', 'yz_kalman', 'yz_gold',
    'zx_kalman', 'zx_gold', 'zy_kalman', 'zy_gold', 'zz_kalman', 'zz_gold',
]]
common_df

,time,x_kalman,x_gold,y_kalman,y_gold,z_kalman,z_gold,xx_kalman,xx_gold,xy_kalman,...,yy_kalman,yy_gold,yz_kalman,yz_gold,zx_kalman,zx_gold,zy_kalman,zy_gold,zz_kalman,zz_gold
0,712215465.00000000,-2575288.61526608,-2575288.61500000,-3682570.76041429,-3682570.76100000,4511064.96091048,4511064.96100000,0.00008221,0.00076427,0.00000015,...,0.00024539,0.00100377,-0.00000018,-0.00000347,-0.00000005,-0.00000229,-0.00000018,-0.00000347,0.00017129,0.00088748
1,712215467.26326096,-2575287.15687353,-2575287.15700000,-3682572.08359653,-3682572.08300000,4511065.66540535,4511065.66500000,0.00019617,0.00031548,0.00000055,...,0.00032609,0.00055488,-0.00000041,-0.00000103,-0.00000024,-0.00000060,-0.00000041,-0.00000103,0.00026478,0.00043844
2,712215467.38339806,-2575287.09816353,-2575287.09800000,-3682572.13760925,-3682572.13700000,4511065.66232908,4511065.66200000,0.00022336,0.00041280,0.00000066,...,0.00034414,0.00065221,-0.00000047,-0.00000144,-0.00000029,-0.00000088,-0.00000047,-0.00000144,0.00028645,0.00053579
3,712215467.40082097,-2575287.08862924,-2575287.08900000,-3682572.14487413,-3682572.14500000,4511065.66016593,4511065.66000000,0.00023008,0.00047584,0.00000069,...,0.00034861,0.00071525,-0.00000049,-0.00000148,-0.00000031,-0.00000091,-0.00000049,-0.00000148,0.00029181,0.00059884
4,712215480.00000000,-2575279.82058940,-2575279.82100000,-3682576.56930882,-3682576.57000000,4511066.24989386,4511066.25000000,0.00008219,0.00076328,0.00000016,...,0.00024529,0.00100290,-0.00000023,-0.00000381,-0.00000007,-0.00000253,-0.00000023,-0.00000381,0.00017124,0.00088686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975,712237109.43815994,-2575994.56153360,-2575994.56200000,-3682881.46810235,-3682881.46800000,4510413.14368385,4510413.14400000,0.00021856,0.00042767,0.00000054,...,0.00025127,0.00049286,-0.00000024,-0.00000110,-0.00000016,-0.00000075,-0.00000024,-0.00000110,0.00026161,0.00051296
1976,712237121.00000000,-2576000.46136819,-2576000.46000000,-3682879.34877182,-3682879.34900000,4510410.88757975,4510410.88700000,0.00009396,0.00078490,0.00000010,...,0.00014097,0.00084321,-0.00000001,-0.00000183,-0.00000000,-0.00000126,-0.00000001,-0.00000183,0.00015629,0.00090805
1977,712237123.29776394,-2576001.69500756,-2576001.69500000,-3682880.92221579,-3682880.92200000,4510410.32310861,4510410.32300000,0.00020154,0.00033179,0.00000069,...,0.00023710,0.00039574,-0.00000038,-0.00000117,-0.00000026,-0.00000079,-0.00000038,-0.00000117,0.00024826,0.00041543
1978,712237123.33963799,-2576001.72548497,-2576001.72500000,-3682880.92364980,-3682880.92400000,4510410.28726605,4510410.28800000,0.00021910,0.00039489,0.00000073,...,0.00025259,0.00045883,-0.00000041,-0.00000130,-0.00000028,-0.00000088,-0.00000041,-0.00000130,0.00026322,0.00047853


In [18]:
common_df[[
    'time','x_kalman', 'x_gold','y_kalman', 'y_gold','z_kalman', 'z_gold',  
]]

,time,x_kalman,x_gold,y_kalman,y_gold,z_kalman,z_gold
0,712215465.00000000,-2575288.61526608,-2575288.61500000,-3682570.76041429,-3682570.76100000,4511064.96091048,4511064.96100000
1,712215467.26326096,-2575287.15687353,-2575287.15700000,-3682572.08359653,-3682572.08300000,4511065.66540535,4511065.66500000
2,712215467.38339806,-2575287.09816353,-2575287.09800000,-3682572.13760925,-3682572.13700000,4511065.66232908,4511065.66200000
3,712215467.40082097,-2575287.08862924,-2575287.08900000,-3682572.14487413,-3682572.14500000,4511065.66016593,4511065.66000000
4,712215480.00000000,-2575279.82058940,-2575279.82100000,-3682576.56930882,-3682576.57000000,4511066.24989386,4511066.25000000
...,...,...,...,...,...,...,...
1975,712237109.43815994,-2575994.56153360,-2575994.56200000,-3682881.46810235,-3682881.46800000,4510413.14368385,4510413.14400000
1976,712237121.00000000,-2576000.46136819,-2576000.46000000,-3682879.34877182,-3682879.34900000,4510410.88757975,4510410.88700000
1977,712237123.29776394,-2576001.69500756,-2576001.69500000,-3682880.92221579,-3682880.92200000,4510410.32310861,4510410.32300000
1978,712237123.33963799,-2576001.72548497,-2576001.72500000,-3682880.92364980,-3682880.92400000,4510410.28726605,4510410.28800000


In [19]:
common_df_comparison = common_df

In [20]:
common_df_comparison["x_kalman_diff_x_gold"] = common_df_comparison["x_kalman"] - common_df_comparison["x_gold"]
common_df_comparison["y_kalman_diff_y_gold"] = common_df_comparison["y_kalman"] - common_df_comparison["y_gold"]
common_df_comparison["z_kalman_diff_z_gold"] = common_df_comparison["z_kalman"] - common_df_comparison["z_gold"]

common_df_comparison[[
    "x_kalman_diff_x_gold", "y_kalman_diff_y_gold", "z_kalman_diff_z_gold"  
]]

,x_kalman_diff_x_gold,y_kalman_diff_y_gold,z_kalman_diff_z_gold
0,-0.00026608,0.00058571,-0.00008952
1,0.00012647,-0.00059653,0.00040535
2,-0.00016353,-0.00060925,0.00032908
3,0.00037076,0.00012587,0.00016593
4,0.00041060,0.00069117,-0.00010614
...,...,...,...
1975,0.00046640,-0.00010235,-0.00031615
1976,-0.00136819,0.00022818,0.00057975
1977,-0.00000756,-0.00021579,0.00010861
1978,-0.00048497,0.00035020,-0.00073395


In [21]:
print(
    f"{common_df_comparison['x_kalman_diff_x_gold'].mean()=}\n"
    f"{common_df_comparison['y_kalman_diff_y_gold'].mean()=}\n"
    f"{common_df_comparison['z_kalman_diff_z_gold'].mean()=}\n"
    f"{common_df_comparison['x_kalman_diff_x_gold'].std()=}\n"
    f"{common_df_comparison['y_kalman_diff_y_gold'].std()=}\n"
    f"{common_df_comparison['z_kalman_diff_z_gold'].std()=}\n"
)

common_df_comparison['x_kalman_diff_x_gold'].mean()=2.391060409747591e-05
common_df_comparison['y_kalman_diff_y_gold'].mean()=2.6766230314593725e-05
common_df_comparison['z_kalman_diff_z_gold'].mean()=-7.101270661811636e-06
common_df_comparison['x_kalman_diff_x_gold'].std()=0.0005834987044181149
common_df_comparison['y_kalman_diff_y_gold'].std()=0.0005731566531924414
common_df_comparison['z_kalman_diff_z_gold'].std()=0.0004917781032406989



In [22]:
common_df[[
    'time',
    'xx_kalman', 'xx_gold',
    'yy_kalman', 'yy_gold',
    'zz_kalman', 'zz_gold',
]]

,time,xx_kalman,xx_gold,yy_kalman,yy_gold,zz_kalman,zz_gold
0,712215465.00000000,0.00008221,0.00076427,0.00024539,0.00100377,0.00017129,0.00088748
1,712215467.26326096,0.00019617,0.00031548,0.00032609,0.00055488,0.00026478,0.00043844
2,712215467.38339806,0.00022336,0.00041280,0.00034414,0.00065221,0.00028645,0.00053579
3,712215467.40082097,0.00023008,0.00047584,0.00034861,0.00071525,0.00029181,0.00059884
4,712215480.00000000,0.00008219,0.00076328,0.00024529,0.00100290,0.00017124,0.00088686
...,...,...,...,...,...,...,...
1975,712237109.43815994,0.00021856,0.00042767,0.00025127,0.00049286,0.00026161,0.00051296
1976,712237121.00000000,0.00009396,0.00078490,0.00014097,0.00084321,0.00015629,0.00090805
1977,712237123.29776394,0.00020154,0.00033179,0.00023710,0.00039574,0.00024826,0.00041543
1978,712237123.33963799,0.00021910,0.00039489,0.00025259,0.00045883,0.00026322,0.00047853


In [23]:
common_df_comparison["xx_kalman_diff_xx_gold"] = common_df_comparison["xx_kalman"] - common_df_comparison["xx_gold"]
common_df_comparison["yy_kalman_diff_yy_gold"] = common_df_comparison["yy_kalman"] - common_df_comparison["yy_gold"]
common_df_comparison["zz_kalman_diff_zz_gold"] = common_df_comparison["zz_kalman"] - common_df_comparison["zz_gold"]

common_df_comparison[[
    "xx_kalman_diff_xx_gold", "yy_kalman_diff_yy_gold", "zz_kalman_diff_zz_gold"  
]]

,xx_kalman_diff_xx_gold,yy_kalman_diff_yy_gold,zz_kalman_diff_zz_gold
0,-0.00068206,-0.00075839,-0.00071619
1,-0.00011930,-0.00022879,-0.00017366
2,-0.00018944,-0.00030807,-0.00024935
3,-0.00024576,-0.00036664,-0.00030703
4,-0.00068109,-0.00075761,-0.00071562
...,...,...,...
1975,-0.00020911,-0.00024159,-0.00025135
1976,-0.00069094,-0.00070224,-0.00075176
1977,-0.00013025,-0.00015864,-0.00016717
1978,-0.00017579,-0.00020624,-0.00021530


In [24]:
print(
    f"{common_df_comparison['xx_kalman_diff_xx_gold'].mean()=}\n"
    f"{common_df_comparison['yy_kalman_diff_yy_gold'].mean()=}\n"
    f"{common_df_comparison['zz_kalman_diff_zz_gold'].mean()=}\n"
    f"{common_df_comparison['xx_kalman_diff_xx_gold'].std()=}\n"
    f"{common_df_comparison['yy_kalman_diff_yy_gold'].std()=}\n"
    f"{common_df_comparison['zz_kalman_diff_zz_gold'].std()=}\n"
)

common_df_comparison['xx_kalman_diff_xx_gold'].mean()=-0.0003384884180906973
common_df_comparison['yy_kalman_diff_yy_gold'].mean()=-0.00036074838240626815
common_df_comparison['zz_kalman_diff_zz_gold'].mean()=-0.000400085795032855
common_df_comparison['xx_kalman_diff_xx_gold'].std()=0.00025666468865550375
common_df_comparison['yy_kalman_diff_yy_gold'].std()=0.0002567939720935288
common_df_comparison['zz_kalman_diff_zz_gold'].std()=0.0002585857262377859



In [25]:
np.allclose(
    common_df["x_kalman"].values, common_df["x_gold"].values,
    rtol=1e-6
)

True

In [26]:
np.allclose(
    common_df["y_kalman"].values, common_df["y_gold"].values,
    rtol=1e-6
)

True

In [27]:
np.allclose(
    common_df["z_kalman"].values, common_df["z_gold"].values,
    rtol=1e-6
)

True

In [28]:
np.allclose(
    common_df["xx_kalman"].values, common_df["xx_gold"].values,
    rtol=1e-5
)

False

In [29]:
np.allclose(
    common_df["yy_kalman"].values, common_df["yy_gold"].values,
    rtol=1e-5
)

False

In [30]:
np.allclose(
    common_df["zz_kalman"].values, common_df["zz_gold"].values,
    rtol=1e-5
)

False